In [3]:
"""
gen = Path(DATA_DIR).glob('*.json')
list(gen)

from os import listdir
listdir('data/pascal')

from os import walk

for dir, sub_dir, fnames in walk(DATA_DIR):
    print(dir)
    print(sub_dir)
    print(fnames)
    break

with open('data/pascal/pascal_train2007.json') as f:
    data = json.load(f)
print(data)

# to test of any object in data annotations has more than 1 bbox
#next((x for x in DATA_ANNOTATIONS if len(x['bbox'])!=4), None)
    
"""    

"\ngen = Path(DATA_DIR).glob('*.json')\nlist(gen)\n\nfrom os import listdir\nlistdir('data/pascal')\n\nfrom os import walk\n\nfor dir, sub_dir, fnames in walk(DATA_DIR):\n    print(dir)\n    print(sub_dir)\n    print(fnames)\n    break\n\nwith open('data/pascal/pascal_train2007.json') as f:\n    data = json.load(f)\nprint(data)\n\n# to test of any object in data annotations has more than 1 bbox\n#next((x for x in DATA_ANNOTATIONS if len(x['bbox'])!=4), None)\n    \n"

In [2]:
from pathlib import Path
import json

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

import pandas as pd

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet34
from torch.utils.data.sampler import SubsetRandomSampler
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import sys
sys.path.append('./PyTorch examples/yunjey/tutorials/04-utils/tensorboard')
from logger import Logger

In [ ]:
from IPython.display import FileLink
FileLink('data/pascal/my-models/largest_item_classifier-89.29_train_acc.ckpt')

In [ ]:
FileLink('./logs/largest_item_classfier/')

# Combined model, classifier + bbox

In [104]:
train_idxs, val_idxs = get_trn_val_idxs(len(largest_item_bbox_csv))

In [105]:
bs = 64
sz = 224
n_epochs = 5
lr = 0.001

classifier_train_tfms = transforms.Compose([
    transforms.Resize((sz, sz)),
    transforms.ToTensor()
])

classifier_val_tfms = transforms.Compose([
    transforms.Resize((sz, sz)),
    transforms.ToTensor()
])

bbox_train_tfms = transforms.Compose([
    ResizeImgAndBbox((sz, sz)),
    RandomFlipImgAndBbox(),
    #RandomRotateImgAndBbox(5), 
    ColorJitter(),
    ToTensor()
])

bbox_val_tfms = transforms.Compose([
    ResizeImgAndBbox((sz, sz)),
    ToTensor()
])

In [145]:
concat_train_ds = ConcatDS(PascalBboxDataset(CSVS_DIR/'largest_item_bbox.csv', JPEG_DIR, bbox_train_tfms), 
                           PascalDataset(CSVS_DIR/'largest_item_classifier.csv', JPEG_DIR, classifier_train_tfms))

concat_train_dl = torch.utils.data.DataLoader(concat_train_ds, 
                                              bs, 
                                              sampler=SubsetRandomSampler(train_idxs))


concat_val_ds = ConcatDS(PascalBboxDataset(CSVS_DIR/'largest_item_bbox.csv', JPEG_DIR, bbox_val_tfms), 
                           PascalDataset(CSVS_DIR/'largest_item_classifier.csv', JPEG_DIR, classifier_val_tfms))

concat_val_dl = torch.utils.data.DataLoader(concat_val_ds, 
                                            bs, 
                                            sampler=SubsetRandomSampler(val_idxs))

In [107]:
custom_head = nn.Sequential(
        Flatten(),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512*7*7, 256),
        nn.ReLU(),
        nn.BatchNorm1d(256),
        nn.Dropout(0.5),
        nn.Linear(256, 4+20)
    )

model = model_utils.get_resnet34_model_with_custom_head(custom_head)

In [ ]:
model.train()
n_epochs = 5
lr = 0.001

logger = Logger('./logs/combined/run_1')
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

step = 0
for e in range(n_epochs):
    for batch, labels in concat_train_dl:
        optimizer.zero_grad()
        
        batch = batch.to(device)
        labels = labels.to(device)
        
        preds = model(batch)
        loss = combined_loss(preds, labels, sz)
        
        loss.backward()
        optimizer.step()
        
        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #
        step +=1
        info = {
            'Detection overall loss': loss.item(), 
            'Detection l1 loss': get_detection_l1_loss(preds[:,:4], labels[0], sz), 
            'Detection cross entropy loss': get_detection_cross_entropy_loss(preds[:,4:], labels[1]),
            'Detection accuracy' : get_detection_accuracy(preds[:,4:], labels[1])
        }
        
        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)
        
        #print loss per 100 steps 
        if (step+1) % 10 == 0:
            print ('Epoch [{}/{}], Iteration [{}/{}], overall Loss: {:.4f}' 
                   .format(e+1, n_epochs, (step+1)%len(concat_train_dl), len(concat_train_dl), loss.item()))

trn_acc, trn_loss = get_concat_model_summary_on_sample_set(model, concat_train_dl, sz)        
val_acc, val_loss = get_concat_model_summary_on_sample_set(model, concat_val_dl, sz)
print('Model accuracy, loss on training set:{}%, {}, validation set:{}%, {}'.format(trn_acc, trn_loss, val_acc, val_loss))                      

In [ ]:
torch.save(model.state_dict(), 'data/pascal/my-models/combined_model_val_77.5.ckpt')

In [ ]:
trn_acc, trn_loss = get_concat_model_summary_on_sample_set(model, concat_train_dl, sz)        
val_acc, val_loss = get_concat_model_summary_on_sample_set(model, concat_val_dl, sz)
print('Model accuracy, loss on training set:{}%, {}, validation set:{}%, {}'.format(trn_acc, trn_loss, val_acc, val_loss))                      

In [1]:
model.eval()
with torch.no_grad():
    batch, labels = iter(concat_val_dl).next()
    
    batch = batch.to(device)
    labels[0] = labels[0].to(device)
    labels[1] = labels[1].to(device)
    
    preds = model(batch)
    
    pred_bboxes, pred_class_scores = preds[:,:4], preds[:, 4:]
    pred_bboxes = torch.sigmoid(pred_bboxes)*sz
    pred_cat_ids = pred_class_scores.argmax(dim=1)
    
    actual_bboxes, actual_cat_ids = labels

NameError: name 'model' is not defined

In [ ]:
fig, axes = plt.subplots(16, 4, figsize=(12*10, 8*10))
for i,ax in enumerate(axes.flat):
    im = batch[i].cpu().numpy().transpose((1,2,0)) 
    ax.imshow(im)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    actual_bbox = yxyx_to_xywh(actual_bboxes[i])
    rect_patch = patches.Rectangle((actual_bbox[0], actual_bbox[1]), actual_bbox[2], actual_bbox[3],fill=False, lw=2, ec='white') 
    ax.add_patch(rect_patch)
    
    pred_bbox = yxyx_to_xywh(pred_bboxes[i])
    pred_bbox = [int(x) for x in pred_bbox]
    pred_rect_patch = patches.Rectangle((pred_bbox[0], pred_bbox[1]), pred_bbox[2], pred_bbox[3],fill=False, lw=2, ec='yellow') 
    ax.add_patch(pred_rect_patch)
    
    pred_cat_label = concat_train_ds.get_category_label(pred_cat_ids[i].item())   
    actual_cat_label = concat_train_ds.get_category_label(actual_cat_ids[i].item())   
    ax.text(0, 0, 'Pred: '+pred_cat_label, size='x-large', weight='bold', color='white', va='top')
    ax.text(0, 30, 'Actual: '+actual_cat_label, size='x-large', weight='bold', color='white', va='top')
    
plt.tight_layout()